Loads relevant packages, instantiates spotify credentials manager

In [6]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import time
import random
import requests
import regex
from string import punctuation
client_id = "6f8bb3e64c2944d586037b9907043601"
client_secret = "81f5f44660364831be8d9118ccb51785"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Setting up the genres and the famous artists from each genre to grab off Spotify

In [40]:
singers = pd.read_csv('singers.csv')
singers

,Singers,Genre
0,Nusrat Fateh Ali Khan,Qawwali
1,Fateh Ali Khan,Qawwali
2,Sabri Brothers,Qawwali
3,Mohammad Rafi,Qawwali
4,Aziz Mian,Qawwali
...,...,...
85,Lil Baby,Hip-Hop
86,Lil Wayne,Hip-Hop
87,DJ Khaled,Hip-Hop
88,Pitbull,Hip-Hop


Getting samples of Top 10 songs from each singer

In [42]:
for i in range(len(singers)):
    artist = singers.loc[i,'Singers']
    result = sp.search(artist)
    singers.loc[i, 'URI'] = result['tracks']['items'][0]['artists'][0]['uri']

Saving to csv so don't have to repeat the process again

In [43]:
singers.to_csv('singersgenreuri.csv')

Getting artist id, track id and preview link for later downloading

In [51]:
artist_uri = []
track_uri = []
track_preview = []
for artist in singers['URI']:
    results = sp.artist_top_tracks(artist)
    for track in results['tracks'][:10]:
        try:  
            artist_uri.append(artist)
            track_uri.append(track['uri'])
            track_preview.append(track['preview_url'])
        except:
            next    

Joining the newly scraped data into a dataframe

In [52]:
track_df = pd.DataFrame()
track_df['Artist'] = artist_uri
track_df['TrackURI'] = track_uri
track_df['Preview'] = track_preview

Saving the data so don't have to download it again

In [53]:
track_df.to_csv('track_df.csv')

Merging the genre/artist data with newly scraped data from Spotify

In [54]:
total_df = pd.merge(singers, track_df, how = 'outer', left_on='URI', right_on = 'Artist')
total_df.to_csv("singers, genre, uri, artist, trackuri.csv")

Removing NAs

In [71]:
total_df2 = total_df.dropna()

Extracting Audio Features from Each Song that Spotify Calculates (Warnimg takes 60min+ to run, better off just full_spotify.csv)

In [77]:
df = pd.DataFrame(total_df2['TrackURI'])
df.columns = ['TrackURI']

for i in range(0,df.shape[0]):
  try:
    time.sleep(random.uniform(3, 6))
    URI = df.TrackURI[i]
    features = sp.audio_features(URI)
    track = sp.track(URI)
    df.loc[i, 'track'] = track['name']
    df.loc[i,'acousticness'] = features[0]['acousticness']
    df.loc[i,'instrumentalness'] = features[0]['instrumentalness']
    df.loc[i,'energy'] = features[0]['energy']
    df.loc[i,'speechiness'] = features[0]['speechiness']
    df.loc[i,'liveness'] = features[0]['liveness']
    df.loc[i,'loudness'] = features[0]['loudness']
    df.loc[i,'danceability'] = features[0]['danceability']
    df.loc[i,'tempo'] = features[0]['tempo']
    df.loc[i,'valence'] = features[0]['valence']
    uri=0
  except:
    next

Saving to CSV

In [82]:
df.to_csv('full_spotify.csv')

Dropping NAs

In [87]:
new_df = df[-df['track'].isna()]

Merging with Earlier Dataset

In [93]:
new_df2 = pd.merge(total_df2, new_df, how = 'left', left_on = 'TrackURI', right_on = 'TrackURI')

Dropping NAs after merge

In [99]:
new_df2 = new_df2[-new_df2['track'].isna()]

Saving to spotify_clean.csv

In [102]:
new_df2.to_csv('spotify_clean.csv')

START FROM HERE

In [24]:
new_df2 = pd.read_csv('spotify_clean.csv')
augmented_dataset = pd.read_csv("augmented_dataset.csv")

Gathering Datsets to Augment

In [28]:
augmented_dataset.drop_duplicates(inplace = True)
complete_spotify = pd.concat([new_df2, augmented_dataset])
pd.DataFrame(complete_spotify['Genre'].value_counts())

,Genre
Classical,3013
Pop,1203
Rap,1047
Blues,747
Rock,655
EDM,594
Hip-Hop,582
Metal,99
Qawwali,89
Ghazal,87


In [29]:
complete_spotify = complete_spotify[-(complete_spotify['Genre'] == 'Jazz')]

In [34]:
size = 87
pick_samples = lambda x: x.sample(size)

subsampled = complete_spotify.groupby('Genre', as_index=False).apply(pick_samples).reset_index(drop=True)

In [36]:
subsampled.to_csv("subsampled.csv")

This downloads previews of all songs in dataframe

In [7]:
for i in range(len(subsampled)):
    url = subsampled.loc[i, "Preview"]
    genre = subsampled.loc[i, "Genre"]
    track_name = subsampled.loc[i, "track"]
    track_name = ' '.join(regex.findall('[A-Za-z0-9]+', track_name))
      
    mp3file = requests.get(url)
    os.makedirs(f'./music/{genre}', exist_ok=True)
    with open(f'./music/{genre}/{str(i).zfill(3)}_{track_name.strip(punctuation)}.mp3','wb') as output:
          output.write(mp3file.content)